# Data Types and Data Wrangling

Prof. Daniel de Abreu Pereira Uhr

### Conteúdo

* Data Types (Tipos de Dados)
  * Dados Numéricos
  * Dados de Sequência
  * Dados Temporais
  * Dados Ausentes
* Data Wrangling (Manipulação de Dados)
  * Classificando e Renomeando
  * Agregando
  * Combinando conjunto de dados
  * Remodelando
  * Função da Janela


### Referências
* [Introduction to Statistical Learning](https://www.statlearning.com/) by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani 
* [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake VanderPlas
* [Python (Documentação Oficial)](https://docs.python.org/3/)
* [NumPy (Array e Computação Numérica)](https://numpy.org/doc/stable/)
* [Pandas (Manipulação e Análise de Dados)](https://pandas.pydata.org/docs/)

# Data Types

Nessa aula usaremos, novamente, os dados *Scraped* do AirBnb para a cidade de Bolonha. Os dados estão disponíveis gratuitamente em *Inside AirBnb* : http://insideairbnb.com/get-the-data.html .

Usaremos 2 conjuntos de dados:

* conjunto de dados de listagem: contém informações de nível de listagem
* conjunto de dados de preços: contém dados de preços ao longo do tempo

In [21]:
import numpy as np
import pandas as pd

In [22]:
# Importando dados de listagens
url_listings = "http://data.insideairbnb.com/italy/emilia-romagna/bologna/2021-12-17/visualisations/listings.csv"
df_listings = pd.read_csv(url_listings)

# Importando dados de preços
url_prices = "http://data.insideairbnb.com/italy/emilia-romagna/bologna/2021-12-17/data/calendar.csv.gz"
df_prices = pd.read_csv(url_prices, compression="gzip")

In [23]:
df_prices.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,1470008,2021-12-17,f,$70.00,$70.00,2,150
1,42196,2021-12-17,f,$68.00,$68.00,3,360
2,42196,2021-12-18,f,$68.00,$68.00,3,360
3,42196,2021-12-19,f,$68.00,$68.00,3,360
4,42196,2021-12-20,f,$68.00,$68.00,3,360


### Dados Numéricos

Métodos

* `+`, `-`, `*`, `/`
* funções numpy
* `pd.cut()`

As operações matemáticas padrão entre colunas são feitas em cada linha do dataframe.

In [24]:
df_prices['maximum_nights'] - df_prices['minimum_nights']

0           148
1           357
2           357
3           357
4           357
           ... 
1260340    1124
1260341    1124
1260342    1124
1260343    1124
1260344    1124
Length: 1260345, dtype: int64

Podemos usar a maioria das operações numpy elemento a elemento em uma única coluna.

In [25]:
np.log(df_listings['price'])

0       4.219508
1       3.367296
2       3.912023
3       4.836282
4       3.912023
          ...   
3448    3.465736
3449    3.806662
3450    3.912023
3451    4.897840
3452    4.744932
Name: price, Length: 3453, dtype: float64

Se quisermos criar essa variável e armazená-la no dataframe, podemos fazer isso diretamente:

In [26]:
df_prices['log_prices'] =  np.log(df_listings['price'])
df_prices['log_prices']

0          4.219508
1          3.367296
2          3.912023
3          4.836282
4          3.912023
             ...   
1260340         NaN
1260341         NaN
1260342         NaN
1260343         NaN
1260344         NaN
Name: log_prices, Length: 1260345, dtype: float64

In [27]:
df_prices.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,log_prices
0,1470008,2021-12-17,f,$70.00,$70.00,2,150,4.219508
1,42196,2021-12-17,f,$68.00,$68.00,3,360,3.367296
2,42196,2021-12-18,f,$68.00,$68.00,3,360,3.912023
3,42196,2021-12-19,f,$68.00,$68.00,3,360,4.836282
4,42196,2021-12-20,f,$68.00,$68.00,3,360,3.912023


Podemos criar variáveis ​​categóricas a partir de uma numérica usando a função `pd.cut()`.

Vejamos o exemplo abaixo:

In [28]:
pd.cut(df_listings['price'], bins = [0, 50, 100, np.inf], labels=['barato', 'regular', 'caro'])

0       regular
1        barato
2        barato
3          caro
4        barato
         ...   
3448     barato
3449     barato
3450     barato
3451       caro
3452       caro
Name: price, Length: 3453, dtype: category
Categories (3, object): ['barato' < 'regular' < 'caro']

`pd.cut()` é uma função do pandas usada para discretizar variáveis numéricas. Ou seja, quebra valores contínuos em categorias.
* `df_listings['price']` : É a série (coluna) de preços que queremos categorizar.
* bins=[0, 50, 100, np.inf]: Define os limites dos intervalos:
  * De 0 até 50
  * De 50 até 100
  * De 100 até o infinito (np.inf)

* `labels=['barato', 'regular', 'caro']`: Define os nomes das categorias para cada intervalo:
  * 0 < preço ≤ 50 → 'barato'
  * 50 < preço ≤ 100 → 'regular'
  * preço > 100 → 'caro'

### Dados de Sequência

Métodos

* "+"
* .str.replace
* .str.contains
* .astype(str) -pd.get_dummies()

Podemos usar o operador `+` entre colunas para fazer acréscimos em pares.

Nota: não podemos fazer isso com strings.

In [29]:
df_listings['host_name']

0                 Carlo
1              Eleonora
2                 Paolo
3            Anna Maria
4               Valerio
             ...       
3448             Ileana
3449           Fernanda
3450             Ileana
3451    Wonderful Italy
3452    Wonderful Italy
Name: host_name, Length: 3453, dtype: object

In [30]:
df_listings['neighbourhood']

0           Santo Stefano
1       Porto - Saragozza
2           Santo Stefano
3           Santo Stefano
4       Porto - Saragozza
              ...        
3448               Navile
3449    Porto - Saragozza
3450               Navile
3451        Santo Stefano
3452    Porto - Saragozza
Name: neighbourhood, Length: 3453, dtype: object

In [31]:
df_listings['host_name'] + df_listings['neighbourhood']

0                     CarloSanto Stefano
1              EleonoraPorto - Saragozza
2                     PaoloSanto Stefano
3                Anna MariaSanto Stefano
4               ValerioPorto - Saragozza
                      ...               
3448                        IleanaNavile
3449           FernandaPorto - Saragozza
3450                        IleanaNavile
3451        Wonderful ItalySanto Stefano
3452    Wonderful ItalyPorto - Saragozza
Length: 3453, dtype: object

Por exemplo, queremos remover o símbolo de dólar da variável `price` no conjunto de dados `df_prices`.

In [32]:
df_prices['price']

0           $70.00
1           $68.00
2           $68.00
3           $68.00
4           $68.00
            ...   
1260340    $115.00
1260341    $115.00
1260342    $115.00
1260343    $115.00
1260344    $115.00
Name: price, Length: 1260345, dtype: object

então, vamos usar o método `.str.replace()` para substituir o símbolo de dólar por uma string vazia.

In [33]:
df_prices['price'].str.replace('$', '', regex=False)

0           70.00
1           68.00
2           68.00
3           68.00
4           68.00
            ...  
1260340    115.00
1260341    115.00
1260342    115.00
1260343    115.00
1260344    115.00
Name: price, Length: 1260345, dtype: object

"regular expression" (*regex*) é uma linguagem especial para buscar, identificar ou substituir padrões em strings.
    * Expressão regular é uma forma de escrever padrões para localizar, substituir ou extrair partes de um texto.

`regex=False` significa que estamos substituindo uma *string literal*, não uma expressão regular.
  * Se True, os padrões de substituição são tratados como expressões regulares. Se False, os padrões serão tratados como strings literais.

Algumas dessas funções usam expressões regulares.

* `match()`: Chame `re.match()` em cada elemento, retornando um booleano.
* `extract()`: Chame `re.match()` em cada elemento, retornando grupos correspondentes como strings.
* `findall()`: Chame `re.findall()` em cada elemento
* `replace()`: Substituir ocorrências de padrão por alguma outra string
* `contains()`: Chame `re.search()` em cada elemento, retornando um booleano
* `count()`: Contar ocorrências do padrão
* `split()`: Equivalente a `str.split()`, mas aceita expressões regulares `rsplit()`

Por exemplo, o próximo código verifica se na palavra **centre** ou **center** estão contidos na descrição do texto.

In [39]:
df_listings['name'].str.contains('centre|center')

0        True
1       False
2        True
3       False
4       False
        ...  
3448    False
3449    False
3450    False
3451    False
3452    False
Name: name, Length: 3453, dtype: bool

Esse código verifica se cada valor da coluna name contém as palavras 'centre' ou 'center', usando uma expressão regular.

* `.str.contains()` → verifica se a string contém um padrão.
* `'centre|center'` → isso é uma expressão regular, onde o símbolo | significa “ou”.
  * Ou seja, ele está buscando:
    * 'centre' OU
    * 'center'

Por fim, podemos (tentar) converter variáveis ​​de string em numéricas usando `astype(float)`.

In [40]:
df_prices['price'].str.replace('[$,]', '', regex=True).astype(float)

0           70.0
1           68.0
2           68.0
3           68.0
4           68.0
           ...  
1260340    115.0
1260341    115.0
1260342    115.0
1260343    115.0
1260344    115.0
Name: price, Length: 1260345, dtype: float64

O que fizemos: 
* `df_prices['price']` acessa a coluna "price" do DataFrame.
  * Essa coluna contém strings como "$1,200.50" ou "$950", por exemplo.
* `.str.replace('[$,]', '', regex=True)`: 
  * Remove os símbolos de dólar ($) e as vírgulas (,) de cada valor.
  * Usa expressão regular para buscar os caracteres dentro de [].
* `.astype(float)`: Converte o valor limpo de string para número com ponto decimal.



Também podemos usá-lo para converter números em strings usando `astype(str)`.

In [41]:
df_listings['id']

0          42196
1          46352
2          59697
3          85368
4         145779
          ...   
3448    53810648
3449    53820830
3450    53837098
3451    53837654
3452    53854962
Name: id, Length: 3453, dtype: int64

repare que dtype é inteiro.

In [42]:
df_listings['id'].astype(str)

0          42196
1          46352
2          59697
3          85368
4         145779
          ...   
3448    53810648
3449    53820830
3450    53837098
3451    53837654
3452    53854962
Name: id, Length: 3453, dtype: object

Agora o dtype é object.

Podemos gerar *dummies* a partir de uma variável categórica usando `pd.get_dummies()`.

In [44]:
df_listings['neighbourhood']

0           Santo Stefano
1       Porto - Saragozza
2           Santo Stefano
3           Santo Stefano
4       Porto - Saragozza
              ...        
3448               Navile
3449    Porto - Saragozza
3450               Navile
3451        Santo Stefano
3452    Porto - Saragozza
Name: neighbourhood, Length: 3453, dtype: object

In [47]:
df_listings['neighbourhood'].value_counts()

Santo Stefano              1195
Porto - Saragozza          1167
San Donato - San Vitale     419
Navile                      403
Borgo Panigale - Reno       146
Savena                      123
Name: neighbourhood, dtype: int64

Vamos criar uma variável identificadora para cada bairro em Bolonha.

In [48]:
pd.get_dummies(df_listings['neighbourhood']).head()

,Borgo Panigale - Reno,Navile,Porto - Saragozza,San Donato - San Vitale,Santo Stefano,Savena
0,0,0,0,0,1,0
1,0,0,1,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,1,0,0,0


### Dados Temporais

Métodos

* `pd.to_datetime()`
* `.dt.year`
* `.df.to_period()`
* `pd.to_timedelta()`

No `df_prices` temos uma variável de data, `date`. Em qual formato ela está? Podemos verificar com o atributo `.dtypes`.

In [49]:
df_prices['date'].dtypes

dtype('O')

In [50]:
df_prices['date']

0          2021-12-17
1          2021-12-17
2          2021-12-18
3          2021-12-19
4          2021-12-20
              ...    
1260340    2022-12-12
1260341    2022-12-13
1260342    2022-12-14
1260343    2022-12-15
1260344    2022-12-16
Name: date, Length: 1260345, dtype: object

Repare que essa variável está como object. Ela apresenta a data no formato "YYYY-MM-DD".

Podemos converter uma variável em uma data usando o método `pd.to_datetime()`.
Vamos adicionar ao dataframe `df_prices` uma nova coluna chamada `datetime` com a data convertida.

In [51]:
df_prices['datetime'] = pd.to_datetime(df_prices['date'])

In [52]:
df_prices[['date', 'datetime']]

,date,datetime
0,2021-12-17,2021-12-17
1,2021-12-17,2021-12-17
2,2021-12-18,2021-12-18
3,2021-12-19,2021-12-19
4,2021-12-20,2021-12-20
...,...,...
1260340,2022-12-12,2022-12-12
1260341,2022-12-13,2022-12-13
1260342,2022-12-14,2022-12-14
1260343,2022-12-15,2022-12-15


De fato, se agora verificarmos o formato da variável `datetime` , é datetime.

In [53]:
df_prices['datetime'].dtypes

dtype('<M8[ns]')

Quando temos uma variável em formato `datetime`, ganhamos muitas operações de data e hora por meio do objeto `dt` para propriedades do tipo data e hora.

Por exemplo, podemos extrair o ano usando `.dt.year`. Podemos fazer o mesmo com `month`, `week` e `day` (se for o caso).

In [54]:
df_prices['datetime'].dt.year

0          2021
1          2021
2          2021
3          2021
4          2021
           ... 
1260340    2022
1260341    2022
1260342    2022
1260343    2022
1260344    2022
Name: datetime, Length: 1260345, dtype: int64

In [55]:
df_prices['datetime'].dt.month

0          12
1          12
2          12
3          12
4          12
           ..
1260340    12
1260341    12
1260342    12
1260343    12
1260344    12
Name: datetime, Length: 1260345, dtype: int64

In [56]:
df_prices['datetime'].dt.day

0          17
1          17
2          18
3          19
4          20
           ..
1260340    12
1260341    13
1260342    14
1260343    15
1260344    16
Name: datetime, Length: 1260345, dtype: int64

Podemos alterar o nível de agregação de uma data usando `.dt.to_period()`. A opção `M` converte para o nível ano-mês.

In [57]:
df_prices['datetime'].dt.to_period('M')

0          2021-12
1          2021-12
2          2021-12
3          2021-12
4          2021-12
            ...   
1260340    2022-12
1260341    2022-12
1260342    2022-12
1260343    2022-12
1260344    2022-12
Name: datetime, Length: 1260345, dtype: period[M]

Podemos adicionar ou subtrair períodos de tempo de uma data usando a função `pd.to_timedelta()`. Precisamos especificar a unidade de medida com a opção `unit`.

In [59]:
df_prices['datetime']

0         2021-12-17
1         2021-12-17
2         2021-12-18
3         2021-12-19
4         2021-12-20
             ...    
1260340   2022-12-12
1260341   2022-12-13
1260342   2022-12-14
1260343   2022-12-15
1260344   2022-12-16
Name: datetime, Length: 1260345, dtype: datetime64[ns]

linha zero apresenta o dia 17. Vamos alterar para 3 dias a menos.

In [60]:
df_prices['datetime'] - pd.to_timedelta(3, unit='d')

0         2021-12-14
1         2021-12-14
2         2021-12-15
3         2021-12-16
4         2021-12-17
             ...    
1260340   2022-12-09
1260341   2022-12-10
1260342   2022-12-11
1260343   2022-12-12
1260344   2022-12-13
Name: datetime, Length: 1260345, dtype: datetime64[ns]

Subtraimos 3 dias de cada valor na coluna 'datetime'

### Dados Ausentes

**Métodos**

* `.isna()`
* `.dropna()`
* `.fillna()`

O método `.isna()` relata valores ausentes.

In [61]:
df_listings.isna().head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True


Para obter uma descrição rápida da quantidade de dados ausentes no conjunto de dados, podemos usar

In [62]:
df_listings.isna().sum()

id                                   0
name                                 0
host_id                              0
host_name                            9
neighbourhood_group               3453
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                        409
reviews_per_month                  409
calculated_host_listings_count       0
availability_365                     0
number_of_reviews_ltm                0
license                           3318
dtype: int64

Podemos remover valores ausentes usando `.dropna()`. Ele **remove todas as linhas com pelo menos um valor ausente**.

In [66]:
df_listings.dropna().shape

(0, 18)

OBS: o resultado "(0, 18)" diz que todas as linhas do seu DataFrame foram removidas — ou seja, todas as linhas têm pelo menos um valor ausente (NaN) em alguma coluna. Veja:

In [67]:
df_listings.dropna().head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license


Neste caso, infelizmente, ele descarta todas as linhas porque quase toda linha tem algum valor faltante. Se quisermos descartar apenas linhas com todos os valores ausentes, podemos usar o parâmetro how='all'.

In [68]:
df_listings.dropna(how='all').shape

(3453, 18)

O novo DataFrame resultante teria: 
* 3453 linhas
* 18 colunas

In [69]:
df_listings.shape

(3453, 18)

exatamente o mesmo que o original.

Se quisermos descartar apenas valores ausentes para um sub-conjunto específico, podemos usar a opção `subset`.

In [21]:
df_listings.dropna(subset=['reviews_per_month']).shape

(3044, 18)

só removeria as linhas onde reviews_per_month está ausente (NaN).

In [71]:
df_listings

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.485070,11.347860,Entire home/apt,68,3,180,2021-11-12,1.32,1,161,6,NaN
1,46352,A room in Pasolini's house,467810,Eleonora,NaN,Porto - Saragozza,44.491680,11.335140,Private room,29,1,300,2021-11-30,2.20,2,248,37,NaN
2,59697,COZY LARGE BEDROOM in the city center,286688,Paolo,NaN,Santo Stefano,44.488170,11.341240,Private room,50,1,240,2020-10-04,2.18,2,327,0,NaN
3,85368,Garden House Bologna,467675,Anna Maria,NaN,Santo Stefano,44.478340,11.356720,Entire home/apt,126,2,40,2019-11-03,0.34,1,332,0,NaN
4,145779,SINGLE ROOM,705535,Valerio,NaN,Porto - Saragozza,44.493060,11.337860,Private room,50,10,69,2021-12-05,0.55,9,365,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,53810648,Camera matrimoniale con bagno condiviso,77662127,Ileana,NaN,Navile,44.510040,11.342230,Private room,32,2,0,NaN,NaN,2,24,0,NaN
3449,53820830,B&B a due passi dalla stazione e da P.zza Magg...,314417328,Fernanda,NaN,Porto - Saragozza,44.504500,11.339930,Private room,45,1,0,NaN,NaN,2,299,0,NaN
3450,53837098,Stanza matrimoniale con bagno privato,77662127,Ileana,NaN,Navile,44.511180,11.342480,Private room,50,1,0,NaN,NaN,2,10,0,NaN
3451,53837654,Casa design all'Antico Ghetto Ebraico by Wonde...,13036400,Wonderful Italy,NaN,Santo Stefano,44.496864,11.344784,Entire home/apt,134,1,0,NaN,NaN,34,94,0,NaN


Também podemos preencher os valores ausentes em vez de descartá-los, usando `fillna()`.

In [72]:
df_listings.fillna(' -- Este estava NaN  -- ')

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,42196,50 sm Studio in the historic centre,184487,Carlo,-- Este estava NaN --,Santo Stefano,44.485070,11.347860,Entire home/apt,68,3,180,2021-11-12,1.32,1,161,6,-- Este estava NaN --
1,46352,A room in Pasolini's house,467810,Eleonora,-- Este estava NaN --,Porto - Saragozza,44.491680,11.335140,Private room,29,1,300,2021-11-30,2.2,2,248,37,-- Este estava NaN --
2,59697,COZY LARGE BEDROOM in the city center,286688,Paolo,-- Este estava NaN --,Santo Stefano,44.488170,11.341240,Private room,50,1,240,2020-10-04,2.18,2,327,0,-- Este estava NaN --
3,85368,Garden House Bologna,467675,Anna Maria,-- Este estava NaN --,Santo Stefano,44.478340,11.356720,Entire home/apt,126,2,40,2019-11-03,0.34,1,332,0,-- Este estava NaN --
4,145779,SINGLE ROOM,705535,Valerio,-- Este estava NaN --,Porto - Saragozza,44.493060,11.337860,Private room,50,10,69,2021-12-05,0.55,9,365,5,-- Este estava NaN --
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,53810648,Camera matrimoniale con bagno condiviso,77662127,Ileana,-- Este estava NaN --,Navile,44.510040,11.342230,Private room,32,2,0,-- Este estava NaN --,-- Este estava NaN --,2,24,0,-- Este estava NaN --
3449,53820830,B&B a due passi dalla stazione e da P.zza Magg...,314417328,Fernanda,-- Este estava NaN --,Porto - Saragozza,44.504500,11.339930,Private room,45,1,0,-- Este estava NaN --,-- Este estava NaN --,2,299,0,-- Este estava NaN --
3450,53837098,Stanza matrimoniale con bagno privato,77662127,Ileana,-- Este estava NaN --,Navile,44.511180,11.342480,Private room,50,1,0,-- Este estava NaN --,-- Este estava NaN --,2,10,0,-- Este estava NaN --
3451,53837654,Casa design all'Antico Ghetto Ebraico by Wonde...,13036400,Wonderful Italy,-- Este estava NaN --,Santo Stefano,44.496864,11.344784,Entire home/apt,134,1,0,-- Este estava NaN --,-- Este estava NaN --,34,94,0,-- Este estava NaN --


Também podemos criar valores ausentes, se quisermos.

repare na linha 2, coluna 2:

In [74]:
df_listings

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.485070,11.347860,Entire home/apt,68,3,180,2021-11-12,1.32,1,161,6,NaN
1,46352,A room in Pasolini's house,467810,Eleonora,NaN,Porto - Saragozza,44.491680,11.335140,Private room,29,1,300,2021-11-30,2.20,2,248,37,NaN
2,59697,COZY LARGE BEDROOM in the city center,286688,Paolo,NaN,Santo Stefano,44.488170,11.341240,Private room,50,1,240,2020-10-04,2.18,2,327,0,NaN
3,85368,Garden House Bologna,467675,Anna Maria,NaN,Santo Stefano,44.478340,11.356720,Entire home/apt,126,2,40,2019-11-03,0.34,1,332,0,NaN
4,145779,SINGLE ROOM,705535,Valerio,NaN,Porto - Saragozza,44.493060,11.337860,Private room,50,10,69,2021-12-05,0.55,9,365,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,53810648,Camera matrimoniale con bagno condiviso,77662127,Ileana,NaN,Navile,44.510040,11.342230,Private room,32,2,0,NaN,NaN,2,24,0,NaN
3449,53820830,B&B a due passi dalla stazione e da P.zza Magg...,314417328,Fernanda,NaN,Porto - Saragozza,44.504500,11.339930,Private room,45,1,0,NaN,NaN,2,299,0,NaN
3450,53837098,Stanza matrimoniale con bagno privato,77662127,Ileana,NaN,Navile,44.511180,11.342480,Private room,50,1,0,NaN,NaN,2,10,0,NaN
3451,53837654,Casa design all'Antico Ghetto Ebraico by Wonde...,13036400,Wonderful Italy,NaN,Santo Stefano,44.496864,11.344784,Entire home/apt,134,1,0,NaN,NaN,34,94,0,NaN


In [75]:
df_listings.iloc[2, 2]

286688

agora vamos alterar por um valor ausente.

In [76]:
df_listings.iloc[2, 2] = np.nan
df_listings

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,42196,50 sm Studio in the historic centre,184487.0,Carlo,NaN,Santo Stefano,44.485070,11.347860,Entire home/apt,68,3,180,2021-11-12,1.32,1,161,6,NaN
1,46352,A room in Pasolini's house,467810.0,Eleonora,NaN,Porto - Saragozza,44.491680,11.335140,Private room,29,1,300,2021-11-30,2.20,2,248,37,NaN
2,59697,COZY LARGE BEDROOM in the city center,NaN,Paolo,NaN,Santo Stefano,44.488170,11.341240,Private room,50,1,240,2020-10-04,2.18,2,327,0,NaN
3,85368,Garden House Bologna,467675.0,Anna Maria,NaN,Santo Stefano,44.478340,11.356720,Entire home/apt,126,2,40,2019-11-03,0.34,1,332,0,NaN
4,145779,SINGLE ROOM,705535.0,Valerio,NaN,Porto - Saragozza,44.493060,11.337860,Private room,50,10,69,2021-12-05,0.55,9,365,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,53810648,Camera matrimoniale con bagno condiviso,77662127.0,Ileana,NaN,Navile,44.510040,11.342230,Private room,32,2,0,NaN,NaN,2,24,0,NaN
3449,53820830,B&B a due passi dalla stazione e da P.zza Magg...,314417328.0,Fernanda,NaN,Porto - Saragozza,44.504500,11.339930,Private room,45,1,0,NaN,NaN,2,299,0,NaN
3450,53837098,Stanza matrimoniale con bagno privato,77662127.0,Ileana,NaN,Navile,44.511180,11.342480,Private room,50,1,0,NaN,NaN,2,10,0,NaN
3451,53837654,Casa design all'Antico Ghetto Ebraico by Wonde...,13036400.0,Wonderful Italy,NaN,Santo Stefano,44.496864,11.344784,Entire home/apt,134,1,0,NaN,NaN,34,94,0,NaN


# Data Wrangling

Aqui continuamos com os dados *Scraped* do AirBnb para a cidade de Bolonha. Os dados estão disponíveis gratuitamente em Inside AirBnb : http://insideairbnb.com/get-the-data.html .

Usaremos, novamente, os 2 conjuntos de dados:

* conjunto de dados de listagem: contém informações de nível de listagem
* conjunto de dados de preços: contém dados de preços ao longo do tempo

In [77]:
import numpy as np
import pandas as pd

In [78]:
# Importando "listings data"
url_listings = "http://data.insideairbnb.com/italy/emilia-romagna/bologna/2021-12-17/visualisations/listings.csv"
df_listings = pd.read_csv(url_listings)

# Importando "pricing data"
url_prices = "http://data.insideairbnb.com/italy/emilia-romagna/bologna/2021-12-17/data/calendar.csv.gz"
df_prices = pd.read_csv(url_prices, compression="gzip")

### Classificando e Renomeando

Você pode classificar os dados usando o método `sort_values`.

Opções
* `ascending`: bool ou lista de bool, padrão True
* `na_position`: {'primeiro', 'último'}, padrão 'último'

In [80]:
df_listings

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.485070,11.347860,Entire home/apt,68,3,180,2021-11-12,1.32,1,161,6,NaN
1,46352,A room in Pasolini's house,467810,Eleonora,NaN,Porto - Saragozza,44.491680,11.335140,Private room,29,1,300,2021-11-30,2.20,2,248,37,NaN
2,59697,COZY LARGE BEDROOM in the city center,286688,Paolo,NaN,Santo Stefano,44.488170,11.341240,Private room,50,1,240,2020-10-04,2.18,2,327,0,NaN
3,85368,Garden House Bologna,467675,Anna Maria,NaN,Santo Stefano,44.478340,11.356720,Entire home/apt,126,2,40,2019-11-03,0.34,1,332,0,NaN
4,145779,SINGLE ROOM,705535,Valerio,NaN,Porto - Saragozza,44.493060,11.337860,Private room,50,10,69,2021-12-05,0.55,9,365,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,53810648,Camera matrimoniale con bagno condiviso,77662127,Ileana,NaN,Navile,44.510040,11.342230,Private room,32,2,0,NaN,NaN,2,24,0,NaN
3449,53820830,B&B a due passi dalla stazione e da P.zza Magg...,314417328,Fernanda,NaN,Porto - Saragozza,44.504500,11.339930,Private room,45,1,0,NaN,NaN,2,299,0,NaN
3450,53837098,Stanza matrimoniale con bagno privato,77662127,Ileana,NaN,Navile,44.511180,11.342480,Private room,50,1,0,NaN,NaN,2,10,0,NaN
3451,53837654,Casa design all'Antico Ghetto Ebraico by Wonde...,13036400,Wonderful Italy,NaN,Santo Stefano,44.496864,11.344784,Entire home/apt,134,1,0,NaN,NaN,34,94,0,NaN


In [82]:
df_listings.sort_values(by=['name', 'price'], ascending=[False, True], na_position='last')

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
2280,38601411,🏡Giardino di Annabella-relax in città-casa intera,240803020,Annabella,NaN,Porto - Saragozza,44.49303,11.31986,Entire home/apt,90,2,53,2021-12-13,1.96,1,76,27,392901
2988,48177313,❤ Romantic Suite with SPA Bath ❤ 4starbologna.com,239491712,4 Star Bologna,NaN,Santo Stefano,44.50271,11.34998,Entire home/apt,309,1,1,2021-03-14,0.11,14,344,1,NaN
3302,52367336,✨House of Alchemy✨,140013413,Greta,NaN,Porto - Saragozza,44.49072,11.30890,Entire home/apt,96,2,7,2021-11-28,3.18,1,88,7,NaN
2039,34495335,♥ Romantic for Couple in Love ♥ | 4 Star Boutique,239491712,4 Star Bologna,NaN,Santo Stefano,44.50368,11.34972,Entire home/apt,143,1,25,2021-08-20,0.79,14,262,6,NaN
2964,47866124,♡Amazing Suite with Private SPA ♡ 4starbologna...,239491712,4 Star Bologna,NaN,Santo Stefano,44.50381,11.34951,Entire home/apt,347,1,2,2021-10-17,0.72,14,337,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2280311,"""Behind Lumieré"" Big & Sunny room in Art District",11610405,Ilaria Silvia,NaN,Porto - Saragozza,44.49868,11.33677,Private room,45,1,220,2021-12-07,2.92,4,83,12,NaN
1040,20702725,"""Al Bulgnén"" Il Bolognino Lovely Flat",148193892,Giuliano,NaN,Savena,44.47824,11.38478,Entire home/apt,60,1,31,2021-02-12,0.60,1,230,1,NaN
1758,31373909,"""Affresco & Arte"" a due passi da Piazza Maggiore",78057000,Adriana,NaN,Santo Stefano,44.48819,11.34081,Entire home/apt,130,2,1,2019-03-18,0.03,1,80,0,NaN
248,4813036,"""A Cà Mì"" in Centro a Bologna",24788923,Prisca,NaN,Santo Stefano,44.49239,11.35322,Entire home/apt,50,3,102,2021-08-31,1.24,1,0,1,NaN


Ou seja, colocamos:
* os nomes em ordem Z → A
* Para nomes iguais, ordena os preços do menor para o maior
* na_position='last' → coloca os valores ausentes por último

Você pode remanejar colunas usando a função `rename()`. Ela recebe um dicionário como argumento `column` no formato {"old_name": "new_name"}.

In [83]:
df_listings.rename(columns={'name': 'listing_name', 'id': 'listing_id'}).head()

,listing_id,listing_name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.48507,11.34786,Entire home/apt,68,3,180,2021-11-12,1.32,1,161,6,NaN
1,46352,A room in Pasolini's house,467810,Eleonora,NaN,Porto - Saragozza,44.49168,11.33514,Private room,29,1,300,2021-11-30,2.20,2,248,37,NaN
2,59697,COZY LARGE BEDROOM in the city center,286688,Paolo,NaN,Santo Stefano,44.48817,11.34124,Private room,50,1,240,2020-10-04,2.18,2,327,0,NaN
3,85368,Garden House Bologna,467675,Anna Maria,NaN,Santo Stefano,44.47834,11.35672,Entire home/apt,126,2,40,2019-11-03,0.34,1,332,0,NaN
4,145779,SINGLE ROOM,705535,Valerio,NaN,Porto - Saragozza,44.49306,11.33786,Private room,50,10,69,2021-12-05,0.55,9,365,5,NaN


### Agregando

Análise exploratória de dados categóricos

Se quisermos contar observações em 2 variáveis ​​categóricas, podemos usar `pd.crosstab()`.

In [28]:
pd.crosstab(df_listings['neighbourhood'], df_listings['room_type'])

room_type,Entire home/apt,Hotel room,Private room,Shared room
neighbourhood,,,,
Borgo Panigale - Reno,107,0,39,0
Navile,250,3,149,1
Porto - Saragozza,842,16,299,10
San Donato - San Vitale,280,1,134,4
Santo Stefano,924,29,237,5
Savena,73,0,48,2


Criamos uma tabela cruzada (ou tabela de contingência) entre:
* Linhas → os valores únicos de `df_listings['neighbourhood']` (bairros)
* Colunas → os valores únicos de `df_listings['room_type']` (tipo de quarto)
* Células → a contagem de ocorrências para cada combinação de bairro e tipo de quarto

Podemos calcular estatísticas por grupo usando `groupby()`.

In [29]:
df_listings.groupby('neighbourhood')[['price', 'reviews_per_month']].mean()

,price,reviews_per_month
neighbourhood,,
Borgo Panigale - Reno,83.020548,0.983488
Navile,142.200993,1.156745
Porto - Saragozza,129.908312,1.340325
San Donato - San Vitale,91.618138,0.933011
Santo Stefano,119.441841,1.344810
Savena,69.626016,0.805888


Se você quiser executar mais de uma função, talvez em colunas diferentes, você pode usar `aggregate()` que pode ser abreviado para `agg()`. 

A sintaxe é `agg(output_var = ("input_var", function))` e aceita também funções `numpy`.

In [84]:
df_listings.groupby('neighbourhood').agg(mean_reviews=("reviews_per_month", "mean"),
                                         min_price=("price", "min"),
                                         max_price=("price", np.max)).reset_index()

,neighbourhood,mean_reviews,min_price,max_price
0,Borgo Panigale - Reno,0.983488,9,1429
1,Navile,1.156745,14,5000
2,Porto - Saragozza,1.340325,7,9999
3,San Donato - San Vitale,0.933011,10,1600
4,Santo Stefano,1.344810,11,9999
5,Savena,0.805888,9,680


Se quisermos construir uma nova coluna por grupo, podemos usar os dados agrupados `transform()` . Infelizmente, não funciona tão bem `aggregate()` e temos que fazer uma coluna de cada vez.

In [86]:
df_listings.groupby('neighbourhood')[['price', 'reviews_per_month']].transform('mean').head(10)

,price,reviews_per_month
0,119.441841,1.344810
1,129.908312,1.340325
2,119.441841,1.344810
3,119.441841,1.344810
4,129.908312,1.340325
5,142.200993,1.156745
6,119.441841,1.344810
7,142.200993,1.156745
8,129.908312,1.340325
9,129.908312,1.340325


ou seja, calcula a média por grupo (bairro) para cada coluna, mas retorna um DataFrame com o mesmo número de linhas do original, preenchendo a média do grupo em cada linha.

### Combinando conjunto de dados

Podemos concatenar *datasets* usando o método `pd.concat()`. Ele recebe como argumento uma lista de dataframes. Por padrão, `pd.concat()` executa o *outer join*. Podemos alterá-lo usando a opção `join` (nesse caso, não faz diferença).

In [87]:
df_listings1 = df_listings[:2000]
np.shape(df_listings1)

(2000, 18)

aqui usamos um *fatiamento* (*slicing*) para selecionar as 2.000 primeiras linhas do DataFrame df_listings

In [89]:
df_listings2 = df_listings[1000:]
np.shape(df_listings2)

(2453, 18)

Aqui um fatiamento diferente, é "praticamente" o complemento da anterior, mas agora estamos pegando da linha 1000 até o fim do DataFrame.

In [90]:
np.shape(pd.concat([df_listings1, df_listings2]))

(4453, 18)

Agora recombinamos os dois subconjuntos (`df_listings1` e `df_listings2`) com `pd.concat()` e verificamos o tamanho com `np.shape()`.

Para mesclar dataframes, podemos usar, também, a função `pd.merge`.

Opções
* how: {'esquerda', 'direita', 'externo', 'interno', 'cruzado'}, padrão 'interno'
* on: rótulo ou lista

In [92]:
df_merged = pd.merge(df_listings, df_prices, left_on='id', right_on='listing_id', how='inner')
df_merged.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price_x,...,availability_365,number_of_reviews_ltm,license,listing_id,date,available,price_y,adjusted_price,minimum_nights_y,maximum_nights
0,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.48507,11.34786,Entire home/apt,68,...,161,6,NaN,42196,2021-12-17,f,$68.00,$68.00,3,360
1,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.48507,11.34786,Entire home/apt,68,...,161,6,NaN,42196,2021-12-18,f,$68.00,$68.00,3,360
2,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.48507,11.34786,Entire home/apt,68,...,161,6,NaN,42196,2021-12-19,f,$68.00,$68.00,3,360
3,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.48507,11.34786,Entire home/apt,68,...,161,6,NaN,42196,2021-12-20,f,$68.00,$68.00,3,360
4,42196,50 sm Studio in the historic centre,184487,Carlo,NaN,Santo Stefano,44.48507,11.34786,Entire home/apt,68,...,161,6,NaN,42196,2021-12-21,f,$68.00,$68.00,3,360


Queremos ligar os dados de preço ao dado do imóvel com base no ID.


* `pd.merge()`: Junta (ou mescla) os dois DataFrames, no caso: `df_listings` e `df_prices`.
* `left_on='id'`, `right_on='listing_id'`: Diz que a junção será feita usando:
  * A coluna `id` do `df_listings` (lado esquerdo)
  * A coluna `listing_id` do df_prices (lado direito)

* `how='inner'`: Isso define o tipo de junção. 
  * `inner` significa: Mantém apenas as linhas que têm correspondência em ambos os DataFrames.
  * Outras opções são: 'left', 'right' e 'outer'.
    * left: Mantém todas as linhas do DataFrame à esquerda e as linhas correspondentes do DataFrame à direita.
    * right: Mantém todas as linhas do DataFrame à direita e as linhas correspondentes do DataFrame à esquerda.
    * outer: Mantém todas as linhas de ambos os DataFrames.

No nosso caso, se um `id` do `df_listings` não existe no `df_prices`, ele será descartado. Se um `listing_id` do `df_prices` não está no `df_listings`, também será descartado.



Outra coisa, como você pode ver, como a variável `price` estava presente em ambos os conjuntos de dados, agora temos a `price_x` e a `price_y`.

### Remodelando

Primeiro, vamos calcular os preços médios `neighbourhood` usando `date` no conjunto de dados mesclado.

In [99]:
df_long = df_merged.groupby(['neighbourhood', 'date'])['price_x'].agg('mean').reset_index()
df_long.head(10)

,neighbourhood,date,price_x
0,Borgo Panigale - Reno,2021-12-17,83.020548
1,Borgo Panigale - Reno,2021-12-18,83.020548
2,Borgo Panigale - Reno,2021-12-19,83.020548
3,Borgo Panigale - Reno,2021-12-20,83.020548
4,Borgo Panigale - Reno,2021-12-21,83.020548
5,Borgo Panigale - Reno,2021-12-22,83.020548
6,Borgo Panigale - Reno,2021-12-23,83.020548
7,Borgo Panigale - Reno,2021-12-24,83.020548
8,Borgo Panigale - Reno,2021-12-25,83.020548
9,Borgo Panigale - Reno,2021-12-26,83.020548


Isso é o que chamamos de **formato longo** (*Long*), pois tem uma ou mais variáveis ​​( `price_x` neste caso) empilhadas verticalmente ao longo de uma variável categórica (`neighborhoode` `date` aqui), que atua como índice.

A alternativa é o **formato amplo** (*Wide*), onde temos uma coluna separada para cada bairro.

Podemos remodelar o conjunto de dados **de longo** para **largo** usando o comando `pd.pivot()`.

In [97]:
df_wide = pd.pivot(data=df_long, index='date', columns='neighbourhood').reset_index()
df_wide.head(10)

date               price_x                                \
neighbourhood             Borgo Panigale - Reno      Navile Porto - Saragozza   
0              2021-12-17             83.020548  142.200993        129.908312   
1              2021-12-18             83.020548  142.200993        129.908312   
2              2021-12-19             83.020548  142.200993        129.908312   
3              2021-12-20             83.020548  142.200993        129.908312   
4              2021-12-21             83.020548  142.200993        129.908312   
5              2021-12-22             83.020548  142.200993        129.908312   
6              2021-12-23             83.020548  142.200993        129.908312   
7              2021-12-24             83.020548  142.200993        129.908312   
8              2021-12-25             83.020548  142.200993        129.908312   
9              2021-12-26             83.020548  142.200993        129.908312   

                                                                
neighbourhood San Donato - San Vitale Santo Stefano     Savena  
0                           91.618138    119.441841  69.626016  
1                           91.618138    119.441841  69.626016  
2                           91.618138    119.441841  69.626016  
3                           91.618138    119.441841  69.626016  
4                           91.618138    119.441841  69.626016  
5                           91.618138    119.441841  69.626016  
6                           91.618138    119.441841  69.626016  
7                           91.618138    119.441841  69.626016  
8                           91.618138    119.441841  69.626016  
9                           91.618138    119.441841  69.626016

Podemos remodelar o conjunto de dados de *wide* para *Long* usando o comando `pd.melt()`. Ele aceita os seguintes argumentos

* data: o dataframe
* id_vars: a variável que estava indexando o antigo conjunto de dados
  

In [100]:
pd.melt(df_wide, id_vars='date', value_name='price').head()

,date,None,neighbourhood,price
0,2021-12-17,price_x,Borgo Panigale - Reno,83.020548
1,2021-12-18,price_x,Borgo Panigale - Reno,83.020548
2,2021-12-19,price_x,Borgo Panigale - Reno,83.020548
3,2021-12-20,price_x,Borgo Panigale - Reno,83.020548
4,2021-12-21,price_x,Borgo Panigale - Reno,83.020548


Se não tivermos colunas `MultiIndex`, mas apenas um prefixo comum, podemos remodelar o conjunto de dados de *Wide* para *Long* usando o comando `pd.wide_to_long()`.

In [101]:
df_wide2 = df_wide.copy()
df_wide2.columns = [''.join(col) for col in df_wide2.columns]
df_wide2.head()

,date,price_xBorgo Panigale - Reno,price_xNavile,price_xPorto - Saragozza,price_xSan Donato - San Vitale,price_xSanto Stefano,price_xSavena
0,2021-12-17,83.020548,142.200993,129.908312,91.618138,119.441841,69.626016
1,2021-12-18,83.020548,142.200993,129.908312,91.618138,119.441841,69.626016
2,2021-12-19,83.020548,142.200993,129.908312,91.618138,119.441841,69.626016
3,2021-12-20,83.020548,142.200993,129.908312,91.618138,119.441841,69.626016
4,2021-12-21,83.020548,142.200993,129.908312,91.618138,119.441841,69.626016


O comando `pd.wide_to_long()` aceita os seguintes argumentos

* data: o dataframe
* stubnames: os prefixos das variáveis ​​que queremos remodelar em uma
* i: a variável que estava indexando o antigo conjunto de dados
* j: o nome da nova variável categórica que extraímos destubnames
* suffix: expressão regular do sufixo, o padrão é \d+, ou seja, dígitos

In [102]:
pd.wide_to_long(df_wide2, stubnames='price_x', i='date', j='neighborhood', suffix='\D+').head()

,,price_x
date,neighborhood,
2021-12-17,Borgo Panigale - Reno,83.020548
2021-12-18,Borgo Panigale - Reno,83.020548
2021-12-19,Borgo Panigale - Reno,83.020548
2021-12-20,Borgo Panigale - Reno,83.020548
2021-12-21,Borgo Panigale - Reno,83.020548


Observe que tivemos que alterar o `suffix` para `\D+`, ou seja, não dígitos.

### Função da Janela

Métodos

* `shift()`
* `expanding()`
* `rolling()`

Quando temos dados de séries temporais, podemos querer fazer operações ao longo do tempo. Primeiro, vamos agregar o conjunto de dados `df_price` no nível ano-mês.

In [103]:
temp = df_prices.copy()
temp['price'] = temp['price'].str.replace('[$|,]', '', regex=True).astype(float)
temp['date'] = pd.to_datetime(temp['date']).dt.to_period('M')
temp = temp.groupby(['listing_id', 'date'])['price'].mean().reset_index()\
    .sort_values(by=['listing_id', 'date'], ascending=[False, True])
temp.head()

,listing_id,date,price
44876,53854962,2021-12,147.400000
44877,53854962,2022-01,137.645161
44878,53854962,2022-02,124.642857
44879,53854962,2022-03,285.096774
44880,53854962,2022-04,115.000000


Podemos adiantar ou atrasar uma variável usando o método `shift()`.

In [104]:
temp['price1'] = temp['price'].shift(1)
temp.head(15)

,listing_id,date,price,price1
44876,53854962,2021-12,147.400000,NaN
44877,53854962,2022-01,137.645161,147.400000
44878,53854962,2022-02,124.642857,137.645161
44879,53854962,2022-03,285.096774,124.642857
44880,53854962,2022-04,115.000000,285.096774
44881,53854962,2022-05,115.000000,115.000000
44882,53854962,2022-06,115.000000,115.000000
44883,53854962,2022-07,115.000000,115.000000
44884,53854962,2022-08,115.000000,115.000000
44885,53854962,2022-09,115.000000,115.000000


Se quisermos adiantar ou atrasar uma variável dentro de um grupo , podemos combinar `shift()` com `groupby()`.

In [106]:
temp['price1'] = temp.groupby('listing_id')['price'].shift(1)
temp.head(10)

,listing_id,date,price,price1
44876,53854962,2021-12,147.400000,NaN
44877,53854962,2022-01,137.645161,147.400000
44878,53854962,2022-02,124.642857,137.645161
44879,53854962,2022-03,285.096774,124.642857
44880,53854962,2022-04,115.000000,285.096774
44881,53854962,2022-05,115.000000,115.000000
44882,53854962,2022-06,115.000000,115.000000
44883,53854962,2022-07,115.000000,115.000000
44884,53854962,2022-08,115.000000,115.000000
44885,53854962,2022-09,115.000000,115.000000


Podemos realizar operações cumulativas usando o método `expanding()`.
Ou seja, teremos um objeto acumulativo que, linha a linha, vai crescendo e acumulando mais valores.

In [107]:
temp['avg_cum_price'] = temp['price'].expanding().mean()
temp.head(10)

,listing_id,date,price,price1,avg_cum_price
44876,53854962,2021-12,147.400000,NaN,147.400000
44877,53854962,2022-01,137.645161,147.400000,142.522581
44878,53854962,2022-02,124.642857,137.645161,136.562673
44879,53854962,2022-03,285.096774,124.642857,173.696198
44880,53854962,2022-04,115.000000,285.096774,161.956959
44881,53854962,2022-05,115.000000,115.000000,154.130799
44882,53854962,2022-06,115.000000,115.000000,148.540685
44883,53854962,2022-07,115.000000,115.000000,144.348099
44884,53854962,2022-08,115.000000,115.000000,141.087199
44885,53854962,2022-09,115.000000,115.000000,138.478479


Para executar operações cumulativas dentro de um grupo , podemos combinar `expanding()` com `groupby()`. Como grupos com observações insuficientes são descartados, precisamos mesclar o conjunto de dados novamente.

In [108]:
temp.groupby('listing_id')['price'].expanding().mean().reset_index(level=0).head(10)

,listing_id,price
0,42196,68.0
1,42196,68.0
2,42196,68.0
3,42196,68.0
4,42196,68.0
5,42196,68.0
6,42196,68.0
7,42196,68.0
8,42196,68.0
9,42196,68.0


Se quisermos realizar uma operação sobre uma janela rolante (móvel) , podemos usar a função `rolling()` com o método `mean()`.

In [109]:
temp['avg3_price'] = temp['price'].rolling(3).mean()
temp.head(10)

,listing_id,date,price,price1,avg_cum_price,avg3_price
44876,53854962,2021-12,147.400000,NaN,147.400000,NaN
44877,53854962,2022-01,137.645161,147.400000,142.522581,NaN
44878,53854962,2022-02,124.642857,137.645161,136.562673,136.562673
44879,53854962,2022-03,285.096774,124.642857,173.696198,182.461598
44880,53854962,2022-04,115.000000,285.096774,161.956959,174.913210
44881,53854962,2022-05,115.000000,115.000000,154.130799,171.698925
44882,53854962,2022-06,115.000000,115.000000,148.540685,115.000000
44883,53854962,2022-07,115.000000,115.000000,144.348099,115.000000
44884,53854962,2022-08,115.000000,115.000000,141.087199,115.000000
44885,53854962,2022-09,115.000000,115.000000,138.478479,115.000000


O que fizemos foi criar uma janela rolante de tamanho 3. Então, o pandas vai olhar 3 linhas por vez ao longo da coluna price.
* `.mean()`: Calcula a média dentro de cada janela (com 3 valores por vez).

### Conclusão

Nesta aula, vimos como o Python e a biblioteca pandas permitem manipular e transformar dados de forma eficiente. Exploramos os principais tipos de dados, operações de conversão, limpeza, ordenação, agregação e combinação de tabelas. Também introduzimos técnicas mais avançadas, como médias móveis e médias acumuladas, que são especialmente úteis para análise de séries temporais. Esses conceitos são fundamentais para a construção de análises econômicas mais robustas e serão utilizados ao longo das próximas aulas.
